# 1-round SPECK 32/32 (FNN)

## Imports

In [44]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pipeline import *

In [45]:
from dataset_rr.make_train_data import make_train_data
from dataset_rr.speck import Speck

## Importing the dataset

In [46]:
n_train_samples = 10**6
n_eval_samples = 10**5
n_rounds = 1

cipher = Speck(n_rounds=n_rounds)

key = cipher.draw_keys(1)

In [47]:
train_samples, train_labels = make_train_data(n_train_samples, cipher, key)
test_samples, test_labels = make_train_data(n_eval_samples, cipher, key)

In [48]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1000000, 32)
===== Label Shape: (32,)
===== Training Samples Shape: (1000000, 32)
===== Sample Shape: (32,)
===== Testing Labels Shape: (100000, 32)
===== Testing Samples Shape: (100000, 32)


## Creating the model

In [49]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [51]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*16

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
learning_rate = 0.1

optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 70
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [52]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 512)               16896     
                                                                 
 dense_43 (Dense)            (None, 512)               262656    
                                                                 
 dense_44 (Dense)            (None, 512)               262656    
                                                                 
 dense_45 (Dense)            (None, 512)               262656    
                                                                 
 dense_46 (Dense)            (None, 512)               262656    
                                                                 
 dense_47 (Dense)            (None, 32)                16416     
                                                                 
Total params: 1,083,936
Trainable params: 1,083,936
No

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [53]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/70
180/180 [==============================] - 28s 152ms/step - loss: 0.1578 - accuracy: 0.0018 - binary_accuracy: 0.7126 - val_loss: 0.1163 - val_accuracy: 6.0000e-05 - val_binary_accuracy: 0.7802
Epoch 2/70
180/180 [==============================] - 27s 150ms/step - loss: 0.1056 - accuracy: 4.7778e-05 - binary_accuracy: 0.8092 - val_loss: 0.0943 - val_accuracy: 1.0000e-05 - val_binary_accuracy: 0.8397
Epoch 3/70
180/180 [==============================] - 26s 147ms/step - loss: 0.0759 - accuracy: 4.3333e-05 - binary_accuracy: 0.8845 - val_loss: 0.0544 - val_accuracy: 1.0000e-04 - val_binary_accuracy: 0.9255
Epoch 4/70
180/180 [==============================] - 26s 143ms/step - loss: 0.0398 - accuracy: 9.4333e-04 - binary_accuracy: 0.9483 - val_loss: 0.0262 - val_accuracy: 0.0022 - val_binary_accuracy: 0.9696
Epoch 5/70
180/180 [==============================] - 25s 140ms/step - loss: 0.0465 - accuracy: 0.0017 - binary_accuracy: 0.9501 - val_loss: 0.0483 - val_accuracy: 0.0058 -

In [54]:
h_loss = history.history['loss']
h_val_loss = history.history['val_loss']
h_val_accuracy = history.history['val_accuracy']

print(h_loss)
print(h_val_loss)
print(h_val_accuracy)

[0.15783682465553284, 0.10555192083120346, 0.07586967200040817, 0.03977176174521446, 0.04646357148885727, 0.041769839823246, 0.03639882430434227, 0.03497932106256485, 0.03466518595814705, 0.03364760801196098, 0.03329853713512421, 0.03298182785511017, 0.032810986042022705, 0.19744260609149933, 0.44386544823646545, 0.44332221150398254, 0.4431290030479431, 0.44302743673324585, 0.44297555088996887, 0.44294825196266174, 0.442905992269516, 0.4428950548171997, 0.4428803324699402, 0.44289273023605347, 0.4428500831127167, 0.4242929220199585, 0.06432530283927917, 0.03807825222611427, 0.03775850683450699, 0.037679169327020645, 0.03742580860853195, 0.03740697354078293, 0.05127374455332756, 0.05297714099287987, 0.05291775241494179, 0.052902091294527054, 0.05286538228392601, 0.0528910867869854, 0.05283598229289055, 0.06988374888896942, 0.05922018736600876, 0.05915306508541107, 0.05913059040904045, 0.059112899005413055, 0.05910145863890648, 0.05909530818462372, 0.05908878892660141, 0.0594262890517711

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [55]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

20/20 [==============================] - 1s 44ms/step - loss: 0.0996 - accuracy: 0.3995 - binary_accuracy: 0.9003
Test loss: 0.09964519739151001
Test accuracy: 0.39945000410079956


In [59]:
save_model(neural_network, '1_round_speck')